In [ ]:
!pip install pandas
!pip install --upgrade torch
!pip install --upgrade transformers
!pip install --upgrade scikit-learn
!pip install --upgrade sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast, RobertaTokenizerFast
import torch

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Change the current working directory to the desired directory in Google Drive
os.chdir('/content/drive/MyDrive/CS224V')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Load the data
# def process_labeled_to_unlabeled():
def process_data_labeling(dates):
  dfs = []
  for date in dates:
      df = pd.read_csv('complex1_preprocessed_' + date + '.csv')
      df = df.dropna(subset=['turn'])
      df['turn'] = df['turn'].astype(int)
      df['directive_product'] = df['directive_product'].astype(int)
      df['utterance'] = df['utterance'].astype(str)

      grouped_df = df.groupby('turn').agg({
          'utterance':' '.join,
          'directive_product': max
      }).reset_index()

      dfs.append(grouped_df)
  return dfs

In [ ]:
dates = ['7.16', '7.30', '8.13', '8.27', '9.10']
dfs = process_data_labeling(dates)
df_concatenated = pd.concat(dfs, ignore_index=True)

In [ ]:
len(df_concatenated)

1558

In [24]:
df_concatenated.head(1000)

,turn,utterance,directive_product
0,1,"alright. So today, I wanted to you know kind o...",0
1,2,Sorry L0 are you sharing your screen?,0
2,3,Yea I am.,0
3,4,"Yeah, yeah I am seeing it",0
4,5,Can anyone see it?,0
...,...,...,...
995,164,I don't think so.,0
996,165,I’ve not seen it.,0
997,166,"Yeah, I haven't seen it either.",0
998,167,"Yeah yea oh yea when I see it, I look for thes...",0


In [25]:
df_concatenated.to_csv('concatenated_transcripts.csv')

In [55]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from transformers import BertTokenizerFast, RobertaTokenizerFast
# import torch

# Initial split: 60% for training, 40% for a temporary set
df_train, df_temp = train_test_split(df_concatenated, train_size=0.6)

# Split the temporary set evenly into the testing and development sets
df_dev, df_test = train_test_split(df_temp, train_size=0.5)

# Split into training and testing sets
train_texts, dev_texts, test_texts, train_labels, dev_labels, test_labels = df_train['utterance'], df_dev['utterance'], df_test['utterance'], df_train['directive_product'], df_dev['directive_product'], df_test['directive_product']

#new edit here
train_labels = train_labels.reset_index(drop=True)
dev_labels = dev_labels.reset_index(drop=True)
test_labels = test_labels.reset_index(drop=True)

# Initialize tokenizers
bert_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
roberta_tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

# Tokenize the texts
train_encodings_bert = bert_tokenizer(list(train_texts), truncation=True, padding=True)
dev_encodings_bert = bert_tokenizer(list(dev_texts), truncation=True, padding=True)
test_encodings_bert = bert_tokenizer(list(test_texts), truncation=True, padding=True)

train_encodings_roberta = roberta_tokenizer(list(train_texts), truncation=True, padding=True)
dev_encodings_roberta = roberta_tokenizer(list(dev_texts), truncation=True, padding=True)
test_encodings_roberta = roberta_tokenizer(list(test_texts), truncation=True, padding=True)

# Create a dataloader
class TwitterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        #  print(f"Accesing index: {idx}")
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create training and testing datasets
train_dataset_bert = TwitterDataset(train_encodings_bert, train_labels)
dev_dataset_bert = TwitterDataset(dev_encodings_bert, dev_labels)
test_dataset_bert = TwitterDataset(test_encodings_bert, test_labels)

train_dataset_roberta = TwitterDataset(train_encodings_roberta, train_labels)
dev_dataset_roberta = TwitterDataset(dev_encodings_roberta, dev_labels)
test_dataset_roberta = TwitterDataset(test_encodings_roberta, test_labels)

In [56]:
from transformers import Trainer, TrainingArguments
def train_model(model, train_dataset, val_dataset, model_path):
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )

    trainer.train()

    # Save the model
    model.save_pretrained(model_path)

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install --upgrade transformers[torch]

In [57]:
from transformers import BertForSequenceClassification, RobertaForSequenceClassification
import os
# os.environ['CUDA_LAUNCH_BLOCKING'] =="1"

# Initialize models
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [58]:
len(train_dataset_bert.labels)

934

In [59]:
for key, value in train_dataset_bert.encodings.items():
  print(f"{key}: {len(value)}")

input_ids: 934
token_type_ids: 934
attention_mask: 934


In [50]:
!ls

 AC_breakdown_processed			    complex1_preprocessed_8.13.csv
 AC_meeting_summaries			    complex1_preprocessed_8.27.csv
 AC_meeting_summaries_norm		    complex1_preprocessed_9.10.csv
 AC_summary_processed			    complex1_summaries.csv
 all_summaries.csv			    complex1_summaries_with_eval.csv
'all_summaries_with_eval (1).gsheet'	    complex1_summaries_with_eval.gsheet
 all_summaries_with_eval.csv		    complex1_v2_9.10_meeting_summary.docx
 all_summaries_with_eval.gsheet		    complex1_v3_9.10_meeting_summary.docx
 Bi-Weekly-Labeled			    complex1_v4_9.10_meeting_summary.docx
 cbm_biweekly_transcripts		    complex1_v5_9.10_meeting_summary.docx
 complex1_8.13_meeting_summary.docx	    complex1_v6_9.10_meeting_summary.docx
 complex1_8.27_meeting_summary.docx	    complex1_v7_9.10_meeting_summary.docx
 complex1_9.10_meeting_summary.docx	    complex1_vs_complex2_summaries.csv
 complex1_convos_coded_7.16.txt		    complex1_vs_complex2_summaries_with_eval.csv
 complex1_convos_coded_7.30.txt		    co

In [60]:
# Call the train_model function for each
model_path = "/content/drive/MyDrive/CS224V/model"
train_model(bert_model, train_dataset_bert, dev_dataset_bert, f'{model_path}/bert_model')
train_model(roberta_model, train_dataset_roberta, dev_dataset_roberta, f'{model_path}/roberta_model')

Step,Training Loss


Step,Training Loss


In [63]:
from transformers import Trainer
# from setup import test_dataset_bert, test_dataset_roberta
# from train import bert_model, roberta_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    em = np.mean([np.array_equal(p,l) for p, l in zip(preds, labels)])
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'exact_match': em
    }

def evaluate_model(model, dataset):
    trainer = Trainer(model, compute_metrics=compute_metrics)
    result = trainer.evaluate(dataset)
    return result

In [41]:
num_labels = len(set(train_labels))
num_labels

2

In [45]:
!ls

 AC_breakdown_processed			    complex1_preprocessed_8.13.csv
 AC_meeting_summaries			    complex1_preprocessed_8.27.csv
 AC_meeting_summaries_norm		    complex1_preprocessed_9.10.csv
 AC_summary_processed			    complex1_summaries.csv
 all_summaries.csv			    complex1_summaries_with_eval.csv
'all_summaries_with_eval (1).gsheet'	    complex1_summaries_with_eval.gsheet
 all_summaries_with_eval.csv		    complex1_v2_9.10_meeting_summary.docx
 all_summaries_with_eval.gsheet		    complex1_v3_9.10_meeting_summary.docx
 Bi-Weekly-Labeled			    complex1_v4_9.10_meeting_summary.docx
 cbm_biweekly_transcripts		    complex1_v5_9.10_meeting_summary.docx
 complex1_8.13_meeting_summary.docx	    complex1_v6_9.10_meeting_summary.docx
 complex1_8.27_meeting_summary.docx	    complex1_v7_9.10_meeting_summary.docx
 complex1_9.10_meeting_summary.docx	    complex1_vs_complex2_summaries.csv
 complex1_convos_coded_7.16.txt		    complex1_vs_complex2_summaries_with_eval.csv
 complex1_convos_coded_7.30.txt		    co

In [52]:
# bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
# roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)

# bert_model.load_state_dict(torch.load('/model/bert_model.pth'))
# roberta_model.load_state_dict(torch.load('/model/roberta_model.pth'))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FileNotFoundError: ignored

In [64]:
from transformers import BertForSequenceClassification, RobertaForSequenceClassification

# Initialize models
# bert_model = BertForSequenceClassification.from_pretrained('models/bert_model')
# roberta_model = RobertaForSequenceClassification.from_pretrained('models/roberta_model')

# Call the evaluate_model function
bert_result = evaluate_model(bert_model, test_dataset_bert)
roberta_result = evaluate_model(roberta_model, test_dataset_roberta)

print("BERT Evaluation Result: ", bert_result)
print("RoBERTa Evaluation Result: ", roberta_result)

BERT Evaluation Result:  {'eval_loss': 0.2373792678117752, 'eval_accuracy': 0.8846153846153846, 'eval_f1': 0.8832218506131551, 'eval_precision': 0.8819415241797913, 'eval_recall': 0.8846153846153846, 'eval_exact_match': 0.8846153846153846, 'eval_runtime': 8.8451, 'eval_samples_per_second': 35.274, 'eval_steps_per_second': 4.409}
RoBERTa Evaluation Result:  {'eval_loss': 0.3081306219100952, 'eval_accuracy': 0.8814102564102564, 'eval_f1': 0.8258528807932555, 'eval_precision': 0.7768840401051939, 'eval_recall': 0.8814102564102564, 'eval_exact_match': 0.8814102564102564, 'eval_runtime': 8.1231, 'eval_samples_per_second': 38.409, 'eval_steps_per_second': 4.801}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
